# Homework 1 (2021) : Data Wrangling and Visualization

Due date : **2021-02-23 @23h55** (this is a **hard deadline**)

## Fill this cell with your names

- OUNNOUGHENE, Mohand Yacine, M1 Informatique  DATA

## Carefully follow instructions given on the [course homepage](https://stephanegaiffas.github.io/big_data_course/homeworks/)

Write in English or French

- The deliverable is a `xxx.ipynb` file (jupyter notebook) or a `xxx.py` file (if you are using `jupytext`) built by completing the template. **We won't execute the code in your notebook:** all your results, displays and plots must be visible without having to rerun everything.

Once again, follow the steps described here : [course homepage](https://stephanegaiffas.github.io/big_data_course/homeworks/). **If you don't: no evaluation!**

## Grading <i class="fa graduation-cap"></i>

Here is the way we'll assess your work

| Criterion | Points | Details |
|:----------|:-------:|:----|
|Spelling and syntax | 3 | English/French  |
|Plots correction | 3 |  Clarity / answers the question  |
|Plot style and cleanliness | 3 | Titles, legends, labels, breaks ... |
|Table wrangling | 4 | ETL, SQL like manipulations |
|Computing Statistics | 5 | SQL `goup by`  and aggregation  |
|DRY compliance | 2 | DRY principle at [Wikipedia](https://en.wikipedia.org/wiki/Don%27t_repeat_yourself)|

If we see a single (or more) `for` loop in your code: **-5 points**.  Everything can be done using high-level `pandas` methods

# Preliminaries

## Notebook: Modus operandi

- This is a [Jupyter Notebook]().
- When you execute code within the notebook, the results appear beneath the code.
- [Jupytext]()

## Packages

- Base `Python` can do a lot. But the full power of `Python` comes from a fast growing collection of `packages`/`modules`.

- Packages are first installed (that is using `pip install` or `conda install`), and if
needed, imported during a session.

- The `docker` image you are supposed to use already offers a lot of packages. You should not need to install new packages.

- Once a package has been installed on your drive, if you want all objects exported by the package to be available in your session, you should import the package, using `from pkg import *`.

- If you just want to pick some sobjects from the package,
you can use qualified names like `pkg.object_name` to access the object (function, dataset, class...)


In [178]:
# importing basic tools
import numpy as np
import pandas as pd

from pandas.api.types import CategoricalDtype

import os            # file operations
import requests as req     # networking

from datetime import date  # if needed

In [179]:
# importing plotting packages
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

In [180]:
# make pandas plotly-friendly
np.set_printoptions(precision=2, suppress=True)
%matplotlib inline
pd.options.plotting.backend = "plotly"

# Getting the data

The data are built and made available by [INSEE](https://www.insee.fr/fr/accueil)  (French Governement Statistics Institute)

Prénoms:
- [https://www.insee.fr/fr/statistiques/fichier/2540004/nat2019_csv.zip](https://www.insee.fr/fr/statistiques/fichier/2540004/nat2019_csv.zip)

This dataset has been growing for a while. It has been considered by
social scientists for decades.  Given names are meant to give insights into a variety
of phenomena, including religious observance.

- A glimpse at the body of work can be found in [_L'archipel français_ by Jérome Fourquet, Le Seuil, 2019 ](https://www.seuil.com/ouvrage/l-archipel-francais-jerome-fourquet/9782021406023)

- Read the [File documentation](https://www.insee.fr/fr/statistiques/2540004?sommaire=4767262#documentation)

## Download the data

**QUESTION:** Download the data into a file which relative path is `'./nat2019_csv.zip'`

__Hints:__

- Have a look at [`requests`](https://requests.readthedocs.io/en/master/).
- Use magic commands to navigate across the file hierarchy and create subdirectories
when needed

In [181]:
params = dict(
    url = 'https://www.insee.fr/fr/statistiques/fichier/2540004/nat2019_csv.zip',
    dirpath = './',
    timecourse = '',
    datafile = 'nat2019.hdf',
    fpath = 'nat2019_csv.zip'
)

In [182]:
# modify location  make sure you are in the right directory
# %cd
# %pwd  #
# %ls
# %mkdir # if needed

In [183]:
url = params['url']      # 'https://www.insee.fr/fr/statistiques/fichier/2540004/nat2019_csv.zip'
fpath = params['fpath']  # './nat2019_csv.zip'

In [184]:
request = req.get(url)
open(fpath, 'wb').write(request.content)

1946257

## Load the data in memory

**QUESTION:** Load the data in a `pandas` `DataFrame` called `data`

__Hints:__

- You should obtain a `Pandas dataframe` with 4 columns.
- Mind the conventions used to build theis `csv` file.
- Package `pandas` provides the convenient tools.
- The dataset, though not too large, is already demanding.
- Don't hesitate to test your methods on a sample of rows method `sample()` from class `DataFrame` can be helpful.

In [185]:
data = pd.read_csv(fpath, sep = ';')
data.sample()

,sexe,preusuel,annais,nombre
535169,2,MAYA,2013,680


## Explore the data

**QUESTION:** Look at the data, Use the attributes `columns`, `dtypes` and the methods `head`, `describe`, to get a feeling of the data.

- This dataset is supposed to report all given names used
for either sex during a year in France since 1900

- The file is made of `652 056` lines and  4 columns.

```
|-- preusuel : object
|-- nombre: int64
|-- sexe: int64
|-- annais: object
```

Each row indicates for a given `preusuel` (prénom usuel, given name), `sexe` (sex), and `annais` (année naissance, birthyear) the `nombre` (number) of babies of the given sex who were given that name during the given year.

|sexe    |preusuel     | annais|   nombre|
|:------|:--------|----:|---:|
|2     |SYLVETTE | 1953| 577|
|1   |BOUBOU   | 1979|   4|
|1   |NILS     | 1959|   3|
|2   |NICOLE   | 2003|  36|
|1   |JOSÉLITO | 2013|   4|


**QUESTION:** Compare memory usage and disk space used by data

**Hints:**

- The method `info`  prints a concise summary of a `DataFrame`.
- With optional parameter `memory_usage`, you can get an estimate
of the amount of memory used by the `DataFrame`.
- Beware that the resulting estimate depends on the argument fed.

In [12]:
data.info(memory_usage = 'deep')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 652056 entries, 0 to 652055
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   sexe      652056 non-null  int64 
 1   preusuel  652054 non-null  object
 2   annais    652056 non-null  object
 3   nombre    652056 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 88.5 MB


**QUESTION:** Display the output of `.describe()` with style.

In [13]:
data.describe()


,sexe,nombre
count,652056.000000,652056.000000
mean,1.544176,131.704768
std,0.498045,893.998555
min,1.000000,1.000000
25%,1.000000,4.000000
50%,2.000000,8.000000
75%,2.000000,25.000000
max,2.000000,53624.000000


**QUESTION:** For each column compute the number of distinct values

In [14]:
data.nunique()


sexe            2
preusuel    34263
annais        121
nombre       7294
dtype: int64

# Transformations

## Improving the data types

**QUESTION:** Make `sexe` a category with two levels `Female` and `Male`. Call the new column `genre`. Do you see any reason why this factor should be ordered?

__Hint:__ Read [Pandas and categorical variables](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html?highlight=category)

In [186]:
def sexeToGenre(sexe):
    return  "M" if (sexe == 1) else "F"

data['genre'] = data['sexe'].astype(CategoricalDtype(ordered=False))
data['genre'] = data['genre'].apply(sexeToGenre)

data

,sexe,preusuel,annais,nombre,genre
0,1,_PRENOMS_RARES,1900,1250,M
1,1,_PRENOMS_RARES,1901,1340,M
2,1,_PRENOMS_RARES,1902,1329,M
3,1,_PRENOMS_RARES,1903,1285,M
4,1,_PRENOMS_RARES,1904,1427,M
...,...,...,...,...,...
652051,2,ZYNEB,2016,6,F
652052,2,ZYNEB,2017,6,F
652053,2,ZYNEB,2018,5,F
652054,2,ZYNEB,2019,7,F


**QUESTION:** Compare memory usage of columns `sexe` and `genre`

In [16]:
data.memory_usage()


Index           128
sexe        5216448
preusuel    5216448
annais      5216448
nombre      5216448
genre        652180
dtype: int64

**QUESTION:** Would it be more memory-efficient to recode `sexe` using modalities `F` and `M` instead of `Male` and `Female` ?

_Insert your answer here_

> For the column 'sexe' for sure, the use of "male"/"female" is better than "1"/"2" but using "F"/"M" instead of "Male"/"Female" has no impact on memory

## Dealing with missing values

**QUESTION:** Variable `annais` class is `object`. Make `annais` of type `float`. Note that missing years are encoded as "XXXX", find a way to deal with that.

__Hint:__  As of releasing this Homework (2021-01-22), `Pandas` is not very good at managing missing values,
see [roadmap](https://pandas.pydata.org/docs/development/roadmap.html). Don't try to convert `annais` into an integer column.

In [187]:
data["annais"] = pd.to_numeric(data['annais'], errors='coerce')
data["annais"] = data["annais"].astype("float")
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 652056 entries, 0 to 652055
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype   
---  ------    --------------   -----   
 0   sexe      652056 non-null  int64   
 1   preusuel  652054 non-null  object  
 2   annais    616192 non-null  float64 
 3   nombre    652056 non-null  int64   
 4   genre     652056 non-null  category
dtypes: category(1), float64(1), int64(2), object(1)
memory usage: 20.5+ MB


## Rename and remove columns

**QUESTION:** Remove useless columns (now that you've created new ones, and rename them). You should end up with a dataframe with columns called `"gender"`, `"year"`, `"count"`, `"firstname`" with the following dtypes:

```python
gender        category
firstname     object
count         int64
year          float64
```

In [188]:
data = data.drop(columns=["sexe"])
data = data.rename(columns={"preusuel":"firstname", "annais":"year", "nombre":"count", "genre": "gender"}) 
data.dtypes


firstname      object
year          float64
count           int64
gender       category
dtype: object

**QUESTION:** How many missing values (NA) have been introduced? How many births are concerned?

In [41]:
data.isnull().sum()


firstname    2
year         0
count        0
gender       0
dtype: int64

**QUESTION:** Read the documentation and describe the origin of rows containing the missing values.

In [40]:
data


,firstname,year,count,gender
0,_PRENOMS_RARES,1900,1250,M
1,_PRENOMS_RARES,1901,1340,M
2,_PRENOMS_RARES,1902,1329,M
3,_PRENOMS_RARES,1903,1285,M
4,_PRENOMS_RARES,1904,1427,M
...,...,...,...,...
652051,ZYNEB,2016,6,F
652052,ZYNEB,2017,6,F
652053,ZYNEB,2018,5,F
652054,ZYNEB,2019,7,F


## Checkpointing: save your transformed dataframe

**QUESTION:** Save the transformed dataframe (retyped and renamed) to `./nat2019_csv.zip`. Try several compression methods.

In [111]:
compression_opts = dict(method='zip', archive_name='nat20191.csv')  
data.to_csv('./nat2019_csv1.zip', index=False, compression = compression_opts) 

/usr/lib/python3.8/zipfile.py:1517: UserWarning:

Duplicate name: 'nat20191.csv'



**QUESTION:** Save the transformed dataframe (retyped and renamed) to `./nat2019.hdf` using `.hdf` format

In [11]:
data.to_hdf('./nat2019.hdf', key='data', format='t')


At that point your working directory should look like:

```
├── hmw_1.py      # if you use `jupytext`
├── hmw_1.ipynb
├── nat2019.hdf
├── nat2019_csv.zip
```

**QUESTION:** Reload the data using `read_hdf(...)` so that the resulting dataframe  is properly typed with meaningful column names.

__Hint:__ use `try: ... except` to handle exceptions such as `FileNotFoundError`

In [112]:
try:
  dataa = pd.read_hdf('./nat2019.hdf')
except:
  print("An exception occurred")

dataa

,sexe,preusuel,annais,nombre
0,1,_PRENOMS_RARES,1900,1250
1,1,_PRENOMS_RARES,1901,1340
2,1,_PRENOMS_RARES,1902,1329
3,1,_PRENOMS_RARES,1903,1285
4,1,_PRENOMS_RARES,1904,1427
...,...,...,...,...
652051,2,ZYNEB,2016,6
652052,2,ZYNEB,2017,6
652053,2,ZYNEB,2018,5
652054,2,ZYNEB,2019,7


## Some data "analytics" and visualization

**QUESTION**: For each year, compute the total number of Female and Male births and the proportion of Female  births among total births

__Hints:__

- Groupby operations using several columns for the groups return a dataframe with a `MultiIndex` index see [Pandas advanced](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html)

- Have a look at `MultiIndex`, `reset_index`, `pivot`, `columns.droplevel`

In [135]:
numberFM = data.groupby(["year", "gender"])["count"].sum().reset_index()
numberFM = numberFM.pivot(index = "year", columns = "gender")

numberFM["female's %"] = (numberFM["count"]["F"] *100) / (numberFM["count"]["F"] + numberFM["count"]["M"])

numberFM


count         female's %
gender       M       F           
year                             
1900    177382  237651  57.260748
1901    195946  257492  56.786595
1902    204338  261437  56.129462
1903    207347  261448  55.770219
1904    214118  264821  55.293263
...        ...     ...        ...
2016    390195  370895  48.732082
2017    382867  364184  48.749550
2018    378045  358730  48.689220
2019    374345  357088  48.820329
XXXX    379296  442181  53.827557

[121 rows x 3 columns]

**QUESTION:** Plot the proportion of female births as a function of year. Don't forget: title, axes labels, ticks, scales, etc.

Because of what we did before, the `plot` method of a `DataFrame` with be rendered using `plotly`, so you can use this. But you can use also `seaborn` or any other available plotting library that you want.

__Hint:__ Mind the missing values in the `year` column

In [119]:
numberFM.plot(y="female's %", title = "Proportion of female births as a function of year [1900 - 2019]")

**QUESTION:** Make any sensible comment about this plot.

_Insert your answer here_

> Nous remarquons une baisse des naissances de femme entre 1900 et 1945 (58% vers 49%) qui se stabilise à partir de l'année 1946 (~ 48%)

**QUESTION:** Explore the fluctuations of sex ratio around its mean value since 1945.
Plot deviations of sex ratio around its mean since 1945 as a function of time.

In [273]:
fig = px.line(numberFM, y = "female's %", title = "1945")
fig.add_scatter(y = numberFM["count"]["F"], name = "F")
fig.add_scatter(y = numberFM["count"]["M"] , name = "M")






# Popular firstnames

## The top-20 most popular firstnames since 1900

**QUESTION:** For each firstname and sex (some names may be given to girls and boys), compute the total number of times this firstname has been given during `1900-2019`. Print the top 20 firstnames given and style your result dataframe using `background_gradient` for instance.

In [57]:
data.groupby(["firstname", "gender"])["count"].sum().sort_values(ascending = False).head(20)


firstname       gender
MARIE           F         2232238
JEAN            M         1914553
PIERRE          M          891243
_PRENOMS_RARES  F          833958
MICHEL          M          820295
_PRENOMS_RARES  M          779966
ANDRÉ           M          711845
JEANNE          F          557640
PHILIPPE        M          538421
LOUIS           M          521429
RENÉ            M          516218
ALAIN           M          506898
JACQUES         M          482714
BERNARD         M          469255
MARCEL          M          468115
DANIEL          M          435281
ROGER           M          423578
ROBERT          M          419098
PAUL            M          419006
CLAUDE          M          411727
Name: count, dtype: int64

## Rare firstnames

**QUESTION:** For each sex, plot the proportion of births given `_PRENOMS_RARES` as a function of the year.

In [310]:
rare = data.where(data["firstname"] == "_PRENOMS_RARES").groupby(["year", "gender"]).sum().reset_index()
rare = rare.pivot(index = "year", columns = "gender")

m = (rare["count"]["M"] *100) / (rare["count"]["F"] + rare["count"]["M"])
f = (rare["count"]["F"] *100) / (rare["count"]["F"] + rare["count"]["M"])

rare["count"]["M"] = m
rare["count"]["F"] = f
rare["male's %"] = rare["count"]["M"]
rare["female's %"] = rare["count"]["F"]



rare = rare.drop(columns=["count"])


fig1 = px.line(rare, y="male's %", title = "1945")
fig1.add_scatter(y = rare["female's %"]["F"], name = "F")
fig1.add_scatter(y = rare["male's %"]["M"] , name = "M")




KeyError: 'count'

# A study of the "Marie" firstname

**QUESTION:** Plot the proportion of female births given name `'MARIE'` or `'MARIE-...'` (compounded names) as a function of the year.
Proceed in such a way that the reader can see the share of compounded names. We are expecting an _area plot_.

__Hints:__

- Have a look at the `.str` accessor (to apply a string method over a whole column containing string)
- Have a look at [r-graph-gallery: stacked area](https://www.r-graph-gallery.com/stacked-area-graph.html)  and
at [ggplot documentation](https://ggplot2.tidyverse.org/reference/geom_ribbon.html). Pay attention on the way you stack the area corresponding to names matching pattern 'MARIE-.*' over or under the are corresponding to babies named 'MARIE'
- See Graphique 3, page 48, de _L'archipel français_  de J. Fourquet. Le Seuil. Essais. Vol. 898.

- Add annotation, 1st World War, Front Populaire, 2nd World War, 1968

In [ ]:
# your code here


# Top 10 firstnames of year 2000

**QUESTION:** For each sex, select the ten most popular names in year 2000, and plot the proportion
of newborns given that name over time. Take into account that some names might have
zero occurrence during certain years.

__Hint:__ Leave aside the rows with '_PRENOMS_RARES'.

In [ ]:
# your code here


# Getting help

- [pandas](https://pandas.pydata.org/pandas-docs/stable/reference/)

- [plotly](https://plotly.com/python/) for animated plots

- [stackoverflow](https://stackoverflow.com)

- [stackoverflow: pandas](https://stackoverflow.com/questions/tagged/pandas)

- [stackoverflow: plotly+python](https://stackoverflow.com/questions/tagged/plotly+python)

- The US `babynames` analogue of the INSEE file has been a playground for data scientists,
 see [https://github.com/hadley/babynames](https://github.com/hadley/babynames)

- Don't Repeat Yourself (DRY) principle  at [Wikipedia](https://en.wikipedia.org/wiki/Don%27t_repeat_yourself)